In [1]:
import pandas as pd 
import numpy as np 
import backtrader as bt
from datetime import datetime
import matplotlib.pyplot as plt
import platform
import seaborn as sns
sns.set_theme(color_codes=True, font='Arial Unicode MS') if platform.system() != "Windows" else sns.set_theme(color_codes=True, font="Microsoft YaHei")
import tushare as ts
pro = ts.pro_api()

# 核心组件
1. 数据加载
2. 交易策略
3. 回测框架设置
4. 运行回测
5. 评估性能

回测数据Lines
至少得有open, high, low, close, volume, openinterest


In [2]:
def get_data(code='600519.SH',starttime='20170101', endtime='20200101'):
    df = pro.daily(ts_code=code, start_date=starttime, end_date=endtime)
    df.index = pd.to_datetime(df.trade_date)
    df["openinterest"] = 0
    # 对df的数据列进行一个整合
    df = df[['open', 'high', 'low', 'close', 'vol', 'openinterest']]
    return df

In [11]:
stock_df = get_data()
stock_df

,open,high,low,close,vol,openinterest
trade_date,,,,,,
2019-12-31,1183.00,1188.00,1176.51,1183.00,22588.81,0
2019-12-30,1170.20,1195.50,1170.20,1185.80,40760.66,0
2019-12-27,1150.00,1172.00,1150.00,1163.00,47066.28,0
2019-12-26,1132.00,1141.98,1119.99,1135.10,22877.27,0
2019-12-25,1143.00,1143.00,1130.01,1133.70,24305.97,0
...,...,...,...,...,...,...
2017-01-09,347.80,352.88,346.54,348.51,35405.00,0
2017-01-06,346.64,359.78,346.10,350.76,68095.62,0
2017-01-05,350.00,351.45,345.44,346.74,41704.48,0


In [14]:
# 加载并读取数据源
fromdate = datetime(2017, 1, 1)
todate = datetime(2020, 1, 1)
data = bt.feeds.PandasData(dataname=stock_df)


# 上穿20日线买入，下穿20日线卖出
class MyStrategy(bt.Strategy):
    params=(
        ('maperiod',20),
    )
    
    def __init__(self) -> None:
        super().__init__()
        self.order = None
        self.ma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)
        print("初始化")
    # 每个bar都会执行一次，回测的每个日期都会执行一次
    def next(self):
        # print(self.ma[0])
        # 判断是否有交易指令正在进行
        if (self.order):
            return
        # 空仓
        if (not self.position): # 如果没有持仓
            if self.datas[0].close[0] > self.ma[0]: # 如果收盘价大于均线
                self.order = self.buy(size=200) # 买入200股
                print("买入")
        else:
            if self.datas[0].close[0] < self.ma[0]: # 如果收盘价小于均线
                self.order = self.sell(size=200)# 卖出200股
                print("卖出") # 打印卖出信息
        pass

# 3. 策略设置
cerebro = bt.Cerebro() # 初始化回测系统
# 将数据加入回测系统 
cerebro.adddata(data) # 将数据传入回测系统
# 加入自己的策略
cerebro.addstrategy(MyStrategy)
# 经纪人
startcash = 100000
cerebro.broker.setcash(startcash)
# 设置手续费
cerebro.broker.setcommission(commission=0.0002)
# 4.回测运行
print(f"初始资金：{startcash}\n回测时间：{fromdate} - {todate}")
cerebro.run()
portval = cerebro.broker.getvalue()
print(f"剩余总资金：{round(portval,2)}\n回测时间：{fromdate} - {todate}")

初始资金：100000
回测时间：2017-01-01 00:00:00 - 2020-01-01 00:00:00
初始化
1154.5855
1151.3355
1148.6955
1146.9955
1149.0305
买入
1151.843
1153.693
1155.331
1157.716
1161.411
1164.6985
1167.8155000000002
1171.7835
1174.8785
1179.5285
1182.7795
1184.6395
1186.6545
1188.4045
1191.983
1195.5165
1199.5665
1202.896
1205.696
1205.9065
1205.599
1205.999
1205.8105
1204.6779999999999
1200.6264999999999
1196.8845
1193.8084999999999
1190.8695
1188.414
1186.0805
1183.3805
1183.8505
1182.9005
1181.3805
1179.062
1176.7525
1175.1574999999998
1172.6779999999999
1172.1654999999998
1171.5149999999999
1171.025
1170.575
1169.1105
1168.414
1168.1505
1167.65
1163.8105
1160.3355
1156.4959999999999
1150.8055
1148.498
1144.774
1142.8985
1141.3685
1139.8690000000001
1139.5285
1138.1115
1137.7115
1134.6290000000001
1131.2845
1128.0345
1123.982
1123.0369999999998
1120.366
1116.416
1112.471
1110.5645
1108.3445
1105.6395
1104.33
1099.035
1093.1405
1084.1174999999998
1075.5014999999999
1066.501
1056.816
1046.9995
1037.62199999999